In [1]:
import pandas as pd

In [2]:
!pip install scrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 271 kB 12.5 MB/s 
     |████████████████████████████████| 57 kB 1.8 MB/s 
     |████████████████████████████████| 93 kB 1.3 MB/s 
     |████████████████████████████████| 261 kB 57.2 MB/s 
     |████████████████████████████████| 3.1 MB 47.4 MB/s 
     |████████████████████████████████| 4.0 MB 66.6 MB/s 
     |████████████████████████████████| 74 kB 3.6 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11959 sha256=5d8bf02c40ecef837bd2e885cd8e54fa706c6facc83d9b3740162d0c1e46be63
  Stored in directory: /root/.cache/pip/wheels/dc/b9/4a/948b1176e084b9e3f85e4ffc3d08f817b1fdf0d973bbb94f81
Successfully built PyDispatcher


In [3]:
!scrapy startproject amazonscrapper

New Scrapy project 'amazonscrapper', using template directory '/usr/local/lib/python3.8/dist-packages/scrapy/templates/project', created in:
    /content/amazonscrapper

You can start your first spider with:
    cd amazonscrapper
    scrapy genspider example example.com


In [4]:
!scrapy genspider amazon_review haha.com

Created spider 'amazon_review' using template 'basic' 


In [5]:
%cd amazonscrapper

/content/amazonscrapper


In [6]:
!scrapy genspider amazon_review haha.com

Created spider 'amazon_review' using template 'basic' in module:
  amazonscrapper.spiders.amazon_review


In [7]:
!scrapy list

amazon_review


In [8]:
%%writefile amazonscrapper/spiders/amazon_review.py

import scrapy

review_url = 'https://www.amazon.com/product-reviews/{}'

product_list_woman_1 = ['B0169P8Y1Q', 'B078RK1RJ2', 'B0823H7DXH', 'B079ZZC81K', 'B081GCF2QC']
product_list_woman_2 = ['B071K9P1MZ', 'B0711CSPNG', 'B07L4DT2HZ', 'B074SZX5K8', 'B016YKKHOI', 'B00SCD250K', 'B07QM6YH1G', 'B07DX77ZV7', 'B016YKL66G', 'B077R7LV2K']

class AmazonReviewSpider(scrapy.Spider):
    name = 'amazon_review'
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
    def start_requests(self):
        for product in product_list_woman_2:
            url = review_url.format(product)
            yield scrapy.Request(url)

    def parse(self, response):
        for review in response.css('[data-hook="review"]'):
            item = {
                'stars': review.css('[data-hook="review-star-rating"] span ::text').get(),
                'title': review.css('[data-hook="review-title"] span ::text').get(),
                'body': review.xpath('normalize-space(.//*[@data-hook="review-body"])').get()
            }
            
            yield item
        
        next_page = response.xpath('//a[text()="Next page"]/@href').get()
        
        if next_page:
            yield scrapy.Request(response.urljoin(next_page))

Overwriting amazonscrapper/spiders/amazon_review.py


In [9]:
!scrapy crawl amazon_review -o review_10_product_women.csv

Streaming output truncated to the last 5000 lines.
{'stars': '3.0 out of 5 stars', 'title': 'Three Stars', 'body': 'a little short but ok'}
2022-12-05 20:10:57 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.com/Hanes-Womens-Jersey-Short-Black/product-reviews/B016YKKHOI/ref=cm_cr_arp_d_paging_btm_489?ie=UTF8&pageNumber=489>
{'stars': '3.0 out of 5 stars', 'title': 'A little short', 'body': 'The shorts are great & comfy but much shorter than shown in picture.'}
2022-12-05 20:10:57 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.com/Hanes-Womens-Jersey-Short-Black/product-reviews/B016YKKHOI/ref=cm_cr_arp_d_paging_btm_489?ie=UTF8&pageNumber=489>
{'stars': '5.0 out of 5 stars', 'title': 'Great fit, I purchased two', 'body': 'I weigh about 140, so I bought a size medium. Great fit, I purchased two !!'}
2022-12-05 20:10:57 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.amazon.com/Womens-Floral-Sleeve-Chiffon-Cardigan/product-reviews/B07DX77ZV7/

In [19]:
data = pd.read_csv('/content/amazonscrapper/review_10_product_women.csv')

In [20]:
data.head()

,stars,title,body
0,4.0 out of 5 stars,Short people and big booty Judy’s.,Honest review would be more like a 4.5. Issue ...
1,4.0 out of 5 stars,Great Pants,I LOVE these pants! I originally bought 2x bec...
2,5.0 out of 5 stars,Kinda big,I usually wear a medium however this was close...
3,5.0 out of 5 stars,OMG I love these pants.,They feel so good and smooth. Great if you are...
4,4.0 out of 5 stars,"Nice, but large","These are comfortable, but as others have ment..."


In [21]:
data.shape

(49802, 3)

In [22]:
data['star'] = data['stars'].apply(lambda x: str(x).split()[0]).astype(float)

In [23]:
data.head()

,stars,title,body,star
0,4.0 out of 5 stars,Short people and big booty Judy’s.,Honest review would be more like a 4.5. Issue ...,4.0
1,4.0 out of 5 stars,Great Pants,I LOVE these pants! I originally bought 2x bec...,4.0
2,5.0 out of 5 stars,Kinda big,I usually wear a medium however this was close...,5.0
3,5.0 out of 5 stars,OMG I love these pants.,They feel so good and smooth. Great if you are...,5.0
4,4.0 out of 5 stars,"Nice, but large","These are comfortable, but as others have ment...",4.0


In [24]:
data['sentiment'] = data['star'].apply(lambda x: 1 if x >= 3 else 0)

In [25]:
data.head()

,stars,title,body,star,sentiment
0,4.0 out of 5 stars,Short people and big booty Judy’s.,Honest review would be more like a 4.5. Issue ...,4.0,1
1,4.0 out of 5 stars,Great Pants,I LOVE these pants! I originally bought 2x bec...,4.0,1
2,5.0 out of 5 stars,Kinda big,I usually wear a medium however this was close...,5.0,1
3,5.0 out of 5 stars,OMG I love these pants.,They feel so good and smooth. Great if you are...,5.0,1
4,4.0 out of 5 stars,"Nice, but large","These are comfortable, but as others have ment...",4.0,1


In [26]:
data['sentiment'].unique()

array([1, 0])

In [27]:
data['sentiment'].value_counts()

1    45180
0     4622
Name: sentiment, dtype: int64